# ISPA - SearchLight
Script adapted from: https://github.com/SylvainTakerkart/inter_subject_pattern_analysis/tree/master/fmri_data

Original paper: https://doi.org/10.1016/j.neuroimage.2019.116205

This script demonstrates how to use the searchlight implementation
available in nilearn to perform group-level decoding using an
inter-subject pattern analysis (ISPA) scheme.

## Importing packages

In [1]:
import pandas as pd
import numpy as np
import os
import os.path as op
import re
import glob
import tarfile
import urllib

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression

from nilearn.image import new_img_like, concat_imgs, clean_img
from nilearn.decoding import SearchLight

import nibabel as nb
from nilearn import plotting

import time  # Import the time module


## Opening files to work with

In [25]:
# Set the path to your GLM directory
data_dir = '/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/PROC_DATA/derivatives/GLM'

# getting a full list of available subjects
files = os.listdir(data_dir)
pattern = r'sub-(\d+)_LSA'
subject_numbers = [re.search(pattern, file_name).group(1) for file_name in files if re.search(pattern, file_name)]

# List of subjects to include, empty means include all
subjects_to_include = ['0194','3142']  # Fill with subject identifiers or leave empty to include all

beta_flist = []
y = []
subj_vect = []
runs = []
        
# Create lists from beta map filenames in 'beta_flist', labels in 'y', and subject numbers in 'subj_vect'.
for subject_number in subject_numbers:
    # If subjects_to_include is empty or subject_number is in the list, process the subject
    if not subjects_to_include or subject_number in subjects_to_include:
        subject_directory = os.path.join(data_dir, f'sub-{subject_number}_LSA')

        # List beta map files for the current subject
        beta_map_files = glob.glob(os.path.join(subject_directory, f'sub-{subject_number}_run*_*.nii.gz'))
        beta_map_files.sort()
        beta_flist.extend(beta_map_files)

        # Extract the label from the beta map filename
        for beta_map_file in beta_map_files:
            label = beta_map_file.split('_')[-3]  # Assumes label is the third last part of the filename
            if "InfOther" in label:  # Adjust labels to consider Own vs. Other (irrelevant to valence)
                label = "Other"
            elif "InfOwn" in label:
                label = "Own"
            y.append(label)
            subj_vect.append(subject_number)

        # Extract run info from the beta map filename (to be used in standardization)
        for beta_map_file in beta_map_files:
            run = beta_map_file.split('_')[-4]  # Assumes run is the fourth last part of the filename
            runs.append(run)

## STANDARDIZATIOIN (per run and subject)

# read image data
print("Reading beta maps from all the subjects...")

fmri_nii_dict = {}

for beta_path in beta_flist:
    parts = re.split(r'[/_]', beta_path)
    subj = parts[-6]
    run = 'run-'+parts[-4]
    condition = parts[-3]
    beta = parts[-2]
    
    image = nb.load(beta_path)
    
    # Check if the subject already exists in the dictionary
    if subj in fmri_nii_dict:
        # If the run exists for the subject, append the data to the existing list
        if run in fmri_nii_dict[subj]:
            fmri_nii_dict[subj][run].append({'condition': condition, 'beta': beta, 'image': image})
        # If the run doesn't exist, create a new entry for the run
        else:
            fmri_nii_dict[subj][run] = [{'condition': condition,'beta': beta, 'image': image}]
    # If the subject doesn't exist, create a new entry for the subject and run
    else:
        fmri_nii_dict[subj] = {run: [{'condition': condition,'beta': beta, 'image': image}]}

# Printing the created dictionary
for subj, subj_value in fmri_nii_dict.items():
    for run, run_value in subj_value.items():
        print(f"Subject: {subj}, Run: {run}")
        for item in run_value:
            print(item)
        print('\n')
        
## CONCATENATE BETAS IMAGES ##

concat_imgs_dict = {}
concat_imgs_stand_dict = {}

# Concatenating the images for the same run and subject
for subj, subj_value in fmri_nii_dict.items():
    for run, run_value in subj_value.items():
        images_to_concat = [item['image'] for item in run_value]
        concatenated_image = concat_imgs(images_to_concat)
        print(f"Subject: {subj}, Run: {run}, Concatenated Beta Images Shape: {concatenated_image.shape}")
        
        # Adding concatenated image to the concat_imgs_dict
        if subj in concat_imgs_dict:
            concat_imgs_dict[subj][run] = concatenated_image
        else:
            concat_imgs_dict[subj] = {run: concatenated_image}

        # Standardized beta maps with clean_img()
        stand_image = clean_img(concatenated_image, standardize=True, detrend=False)
        print(f"Subject: {subj}, Run: {run}, Concatenated Standardized Beta Images Shape: {stand_image.shape}")
        
        # Adding standardized image to the concat_imgs_stand_dict
        if subj in concat_imgs_stand_dict:
            concat_imgs_stand_dict[subj][run] = stand_image
        else:
            concat_imgs_stand_dict[subj] = {run: stand_image}
            
# Concatenate all subjects #

all_images_to_concat = []

for subj, subj_value in concat_imgs_stand_dict.items():
    for run, stand_image in subj_value.items():
        all_images_to_concat.append(stand_image)

all_betas_stand = concat_imgs(all_images_to_concat) 

print(f"All Betas Stand Shape: {all_betas_stand.shape}")

Reading beta maps from all the subjects...
Subject: sub-0194, Run: run-01
{'condition': 'InfOtherNeg', 'beta': '01', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be2dca90>}
{'condition': 'InfOtherNeg', 'beta': '02', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be285a30>}
{'condition': 'InfOtherNeg', 'beta': '03', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be285b50>}
{'condition': 'InfOtherNeg', 'beta': '04', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be2852b0>}
{'condition': 'InfOtherNeg', 'beta': '05', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be296580>}
{'condition': 'InfOtherNeg', 'beta': '06', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be296910>}
{'condition': 'InfOtherPos', 'beta': '01', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be2fe2b0>}
{'condition': 'InfOtherPos', 'beta': '02', 'image': <nibabel.nifti1.Nifti1Image object at 0x7f30be2fe7f0>}
{'condition': 'InfOtherPos', 'beta': '03', 'image': <nibabel.nifti1.Ni

/dados/USERS/tiago.bortolini/.local/lib/python3.8/site-packages/nilearn/image/image.py:1272: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  data = signal.clean(


Subject: sub-0194, Run: run-01, Concatenated Standardized Beta Images Shape: (63, 74, 49, 24)
Subject: sub-0194, Run: run-02, Concatenated Beta Images Shape: (63, 74, 49, 24)
Subject: sub-0194, Run: run-02, Concatenated Standardized Beta Images Shape: (63, 74, 49, 24)
Subject: sub-3142, Run: run-01, Concatenated Beta Images Shape: (63, 74, 49, 24)
Subject: sub-3142, Run: run-01, Concatenated Standardized Beta Images Shape: (63, 74, 49, 24)
Subject: sub-3142, Run: run-02, Concatenated Beta Images Shape: (63, 74, 49, 23)
Subject: sub-3142, Run: run-02, Concatenated Standardized Beta Images Shape: (63, 74, 49, 23)
All Betas Stand Shape: (63, 74, 49, 95)


# Adjust Masks 

In [26]:
from nilearn.image import math_img, new_img_like, concat_imgs, resample_to_img

# reading brain mask
mask_brain = nb.load("/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/brain_mask.nii.gz")

# Check whether ROI and Betas are in the same space and resample if necessary
beta_img = nb.load(beta_map_files[0])
if not beta_img.shape == mask_brain.shape:
    resampled_mask_brain = resample_to_img(mask_brain, beta_img)
print("Shape of original Beta image: %s" % (beta_img.shape,))
print("Shape of original ROI image: %s" % (mask_brain.shape,))
print("Shape of resampled ROI image: %s" % (resampled_mask_brain.shape,))

/dados/USERS/tiago.bortolini/.local/lib/python3.8/site-packages/nilearn/image/resampling.py:291: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(


Shape of original Beta image: (63, 74, 49)
Shape of original ROI image: (91, 109, 91)
Shape of resampled ROI image: (63, 74, 49)


In [27]:
from nilearn.image import math_img, new_img_like, concat_imgs, resample_to_img
roi_mask_nii = nb.load("/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/BIDS_Nilearn/2ndLvPilotSPACE_Binar_NAccBrian_OFC_Septal.nii")

# Check whether ROI and Betas are in the same space and resample if necessary
beta_img = nb.load(beta_map_files[0])
if not beta_img.shape == roi_mask_nii.shape:
    resampled_ROI = resample_to_img(roi_mask_nii, beta_img)
print("Shape of original Beta image: %s" % (beta_img.shape,))
print("Shape of original ROI image: %s" % (roi_mask_nii.shape,))
print("Shape of resampled ROI image: %s" % (resampled_ROI.shape,))

/dados/USERS/tiago.bortolini/.local/lib/python3.8/site-packages/nilearn/image/resampling.py:291: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(


Shape of original Beta image: (63, 74, 49)
Shape of original ROI image: (97, 115, 97)
Shape of resampled ROI image: (63, 74, 49)


# Setting up for leave-one-subject-out cross-validation 

In [28]:
# set up leave-one-subject-out cross-validation
loso = LeaveOneGroupOut()
unique_subjects = np.unique(subj_vect)
n_splits = loso.get_n_splits(groups=unique_subjects)

In [ ]:
# output directory
output_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut"
os.makedirs(output_dir, exist_ok=True)

# Calculate the chance level
chance_level = 1. / len(np.unique(y))

# running searchlight decoding
searchlight_radius = 4
n_jobs = -1
y = np.array(y)
single_split_path_list = []
print("Launching cross-validation...")
for split_ind, (train_inds,test_inds) in enumerate(loso.split(subj_vect,subj_vect,subj_vect)):
    print("...split {:02d} of {:02d}".format(split_ind+1, n_splits))
    single_split = [(train_inds,test_inds)]
    y_train = y[train_inds]
    n_samples = len(y_train)
    class_labels = np.unique(y_train)
    clf = LogisticRegression()
    searchlight = SearchLight(resampled_mask_brain,
                              #process_mask_img=resampled_ROI,
                              radius=searchlight_radius,
                              n_jobs=n_jobs,
                              verbose=1,
                              cv=single_split,
                              estimator=clf)
    print("...mapping the data (this takes a long time) and fitting the model in each sphere")
    searchlight.fit(all_betas_stand, y)

    single_split_nii = new_img_like(resampled_mask_brain,searchlight.scores_ - chance_level)
    single_split_path = op.join(output_dir, 'ispa_searchlight_accuracy_split{:02d}of{:02d}.nii'.format(split_ind+1,n_splits))
    print('Saving score map for cross-validation fold number {:02d}'.format(split_ind+1))
    single_split_nii.to_filename(single_split_path)
    single_split_path_list.append(single_split_path)

# Trying to do Non-parametric Permutations with nilearn
Originally it was performed with SnPM MATLAB toolbox to analyse the single-fold (for the inter-subject cross-validation of ISPA) accuracy maps, with 1000 permutations and a significance threshold (p<0,05, FWE corrected).
Here, we are trying to do the same, but using nilearn.

Original SnPM batch: https://github.com/SylvainTakerkart/inter_subject_pattern_analysis/blob/master/fmri_data/snpm_batch.m

## Here we have tried to implement the permutatoin scheme suggested by Stelzer et al. 2013, by permuting labels for X searchlights per subject

In [30]:
# output directory
output_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut"
os.makedirs(output_dir, exist_ok=True)

# Calculate the chance level
chance_level = 1. / len(np.unique(y))

# running searchlight decoding
searchlight_radius = 4
n_jobs = -1
y = np.array(y)
single_split_path_list = []

# Number of permutations within subject
n_permutations = 100

# List to hold the timings
timings = []

# List to hold the mean accuracies per permutation and split
mean_accs = []

print("Launching cross-validation...")
total_start_time = time.time()  # Capture the start time of the entire process

for split_ind, (train_inds,test_inds) in enumerate(loso.split(subj_vect,subj_vect,subj_vect)):
    print("...split {:02d} of {:02d}".format(split_ind+1, n_splits))
    split_start_time = time.time()  # Start time for this split
    
    for permutation in range(n_permutations):
        print(f"...within-subject permutation {permutation+1:02d} of {n_permutations}")
        permutation_start_time = time.time()  # Start time for this permutation
        
        # Append the results to timing list
        #timings.append({f"Start time for split {split_ind+1}': split_start_time})
        
        # Shuffle y labels for this permutation
        np.random.seed(permutation)  # Setting the seed for reproducibility
        y_shuffled = np.random.permutation(y)

        # Set up the classifier and searchlight with the shuffled labels
        clf = LogisticRegression()
        searchlight = SearchLight(resampled_mask_brain,
                                  radius=searchlight_radius,
                                  n_jobs=n_jobs,
                                  verbose=1,
                                  cv=[(train_inds, test_inds)],
                                  estimator=clf)
        
        searchlight_start_time = time.time()  # Start time for this searchlight
        # Fit the model with shuffled labels
        searchlight.fit(all_betas_stand, y_shuffled)

        # Compute accuracy map and subtract chance level
        accuracy_map = searchlight.scores_ - chance_level

        # Save the accuracy map for this permutation and split
        single_split_nii = new_img_like(resampled_mask_brain, accuracy_map)
        output_filename = f'split_{split_ind+1:02d}_permutation_{permutation+1:03d}_accuracy.nii'
        single_split_path = os.path.join(output_dir, output_filename)
        print(f'Saving permutation score map {permutation+1:03d} for split {split_ind+1:02d}')
        single_split_nii.to_filename(single_split_path)
        
        # Append the split number, permutation number, and mean accuracy to mean_accs
        mean_accs.append({
            "Split Number": split_ind + 1,
            "Permutation Number": permutation + 1,
            "Mean Accuracy": (np.mean(accuracy_map))
        })
        
        # Print duration for this search light
        searchlight_end_time = time.time()
    
        print(f"Searchlight {permutation+1} completed in {searchlight_end_time - searchlight_start_time:.2f} seconds.")
        # Append the results to timing list
        timings.append({f"Searchlight {permutation+1}: {searchlight_end_time - searchlight_start_time:.2f}"})
        
    # Print duration for this split
    split_end_time = time.time()
    print(f"Split {split_ind+1} completed in {split_end_time - split_start_time:.2f} seconds.")
    timings.append({f"Split {split_ind+1}: {split_end_time - split_start_time:.2f}"})

# Print the total duration
total_end_time = time.time()
print(f"Total processing time: {total_end_time - total_start_time:.2f} seconds.")
timings.append({f"Total processing time: {total_end_time - total_start_time:.2f}"})

new_df = pd.DataFrame(timings)
new_df.to_csv('timings_all.csv', index=False)

# Create a DataFrame from mean_accs
mean_accs_df = pd.DataFrame(mean_accs)

# Save to CSV
mean_accs_csv_path = os.path.join(output_dir, "null_array_100_perm.csv")
mean_accs_df.to_csv(mean_accs_csv_path, index=False)

Launching cross-validation...
...split 01 of 02
...within-subject permutation 01 of 2


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
Job #1, processed 0/1694 voxels (0.00%, 398.07796478271484 seconds remaining)
Job #1, processed 10/1694 voxels (0.59%, 25.004716279142993 seconds remaining)
Job #1, processed 20/1694 voxels (1.18%, 21.40457630965669 seconds remaining)
Job #1, processed 30/1694 voxels (1.77%, 20.12492130974592 seconds remaining)
Job #1, processed 40/1694 voxels (2.36%, 19.837697190753488 seconds remaining)
Job #1, processed 50/1694 voxels (2.95%, 19.785327551728585 seconds remaining)
Job #1, processed 60/1694 voxels (3.54%, 19.402810479288046 seconds remaining)
Job #1, processed 70/1694 voxels (4.13%, 18.29700283565475 seconds remaining)
Job #1, processed 80/1694 voxels (4.72%, 17.50501993551093 seconds remaining)
Job #1, processed 90/1694 voxels (5.31%, 16.926088043525397 seconds remaining)
Job #1, processed 100/1694 voxels (5.90%, 16.338840355307365 seconds remaining)
Job #1, processed 110/1694 voxels (6.49%, 15.7982344664116

Saving permutation score map 001 for split 01
Searchlight 1 completed in 91.09 seconds.
...within-subject permutation 02 of 2


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
Job #14, processed 1000/1694 voxels (59.03%, 5.59171332320217 seconds remaining)
Job #14, processed 1010/1694 voxels (59.62%, 5.501864279728135 seconds remaining)
Job #14, processed 1020/1694 voxels (60.21%, 5.422300129945721 seconds remaining)
Job #14, processed 1030/1694 voxels (60.80%, 5.335930661151284 seconds remaining)
Job #14, processed 1040/1694 voxels (61.39%, 5.255353962060468 seconds remaining)
Job #14, processed 1050/1694 voxels (61.98%, 5.172830436798249 seconds remaining)
Job #14, processed 1060/1694 voxels (62.57%, 5.090999898084422 seconds remaining)
Job #14, processed 1070/1694 voxels (63.16%, 5.007714062721538 seconds remaining)
Job #14, processed 1080/1694 voxels (63.75%, 4.923735562492819 seconds remaining)
Job #14, processed 1090/1694 voxels (64.34%, 4.841406305124568 seconds remaining)
Job #14, processed 1100/1694 voxels (64.94%, 4.754926895852966 seconds remaining)
Job #14, processed 111

Saving permutation score map 002 for split 01
Searchlight 2 completed in 85.08 seconds.
Split 1 completed in 176.17 seconds.
...split 02 of 02
...within-subject permutation 01 of 2


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
Job #18, processed 1290/1694 voxels (76.15%, 3.129688930918492 seconds remaining)
Job #18, processed 1300/1694 voxels (76.74%, 3.052872385741338 seconds remaining)
Job #18, processed 1310/1694 voxels (77.33%, 2.9742110880229324 seconds remaining)
Job #18, processed 1320/1694 voxels (77.92%, 2.896985517145427 seconds remaining)
Job #18, processed 1330/1694 voxels (78.51%, 2.8175298089571714 seconds remaining)
Job #18, processed 1340/1694 voxels (79.10%, 2.7402720674401446 seconds remaining)
Job #18, processed 1350/1694 voxels (79.69%, 2.662508252394336 seconds remaining)
Job #18, processed 1360/1694 voxels (80.28%, 2.584474496361505 seconds remaining)
Job #18, processed 1370/1694 voxels (80.87%, 2.507178626308721 seconds remaining)
Job #18, processed 1380/1694 voxels (81.46%, 2.429866639572077 seconds remaining)
Job #18, processed 1390/1694 voxels (82.05%, 2.3515585285363443 seconds remaining)
Job #18, processe

Saving permutation score map 001 for split 02
Searchlight 1 completed in 85.66 seconds.
...within-subject permutation 02 of 2


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
Job #17, processed 1600/1694 voxels (94.45%, 0.7376628002835175 seconds remaining)
Job #17, processed 1610/1694 voxels (95.04%, 0.6594211573552595 seconds remaining)
Job #17, processed 1620/1694 voxels (95.63%, 0.5810165725890711 seconds remaining)
Job #17, processed 1630/1694 voxels (96.22%, 0.5026334230529621 seconds remaining)
Job #17, processed 1640/1694 voxels (96.81%, 0.42414229929096203 seconds remaining)
Job #17, processed 1650/1694 voxels (97.40%, 0.3455910016868631 seconds remaining)
Job #17, processed 1660/1694 voxels (97.99%, 0.26722333345842475 seconds remaining)
Job #17, processed 1670/1694 voxels (98.58%, 0.18880619483246597 seconds remaining)
Job #17, processed 1680/1694 voxels (99.17%, 0.11014067026784324 seconds remaining)
Job #17, processed 1690/1694 voxels (99.76%, 0.031855593706572344 seconds remaining)
Job #5, processed 0/1694 voxels (0.00%, 72.38149642944336 seconds remaining)
Job #5, pr

Saving permutation score map 002 for split 02
Searchlight 2 completed in 85.75 seconds.
Split 2 completed in 171.42 seconds.
Total processing time: 347.59 seconds.


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   14.0s finished
Job #1, processed 980/1694 voxels (57.85%, 5.391539457860236 seconds remaining)
Job #1, processed 990/1694 voxels (58.44%, 5.317601603077831 seconds remaining)
Job #1, processed 1000/1694 voxels (59.03%, 5.244005953156906 seconds remaining)
Job #1, processed 1010/1694 voxels (59.62%, 5.1679556911242015 seconds remaining)
Job #1, processed 1020/1694 voxels (60.21%, 5.094266308718119 seconds remaining)
Job #1, processed 1030/1694 voxels (60.80%, 5.018418936352981 seconds remaining)
Job #1, processed 1040/1694 voxels (61.39%, 4.946402823542477 seconds remaining)
Job #1, processed 1050/1694 voxels (61.98%, 4.869604939989753 seconds remaining)
Job #1, processed 1060/1694 voxels (62.57%, 4.794341616914813 seconds remaining)
Job #1, processed 1070/1694 voxels (63.16%, 4.718103853791028 seconds remaining)
Job #1, processed 1080/1694 voxels (63.75%, 4.644647299074659 seconds remaining)
Job #1, processed 1090/1694 voxels (64.

## Randomly select one accuracy map per subject and average (repeat 10n5 x).

## Pool of 10n5 chance group accuracy maps


In [ ]:
           
import numpy as np
import nibabel as nib
import os
import glob
from nilearn.image import new_img_like

# Define input and output directories
input_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut"

# Function to extract unique split identifiers based on existing files in directory
def get_unique_identifiers(directory, pattern):
    files = glob.glob(os.path.join(directory, pattern))
    unique_ids = set()
    for file in files:
        match = re.search(r"split_(\d+)_permutation_(\d+)_accuracy\.nii", file)
        if match:
            unique_ids.add(match.group(1))
    return list(unique_ids)

# Use the function to get a list of unique split identifiers
split_identifiers = get_unique_identifiers(input_dir, "split_*_permutation_*_accuracy.nii")

# fix identifier for testing
#split_identifiers= ['01','02','03']

# Preload maps paths for each split identifier
split_maps_paths = {}
for split_identifier in split_identifiers:
    pattern = os.path.join(input_dir, f"split_{split_identifier}_permutation_*_accuracy.nii")
    split_maps_paths[split_identifier] = glob.glob(pattern)


In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import random
import csv

n_iterations = 100000
output_dir = os.path.join(input_dir, "permu_mean_group_acc_maps")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load an example NIfTI image to get the shape and affine
example_img_path = split_maps_paths[split_identifiers[0]][0]
example_img = nib.load(example_img_path)
img_shape = example_img.shape
affine = example_img.affine

def perform_iteration(iteration, output_dir):
    start_time = time.time()  # Start timing the iteration
    #print(f"Starting iteration {iteration+1}")  # Debugging line
    sum_of_maps = np.zeros(img_shape)
    selected_maps = []  # Initialize a list to store the paths of selected maps
    
    # Randomly select one map per subject/split and accumulate
    for split_id, map_paths in split_maps_paths.items():
        selected_map_path = random.choice(map_paths)
        selected_maps.append(selected_map_path)  # Store the selected map path
        #print(f"Selected map for {split_id}: {selected_map_path}")  # Debugging line
        map_data = nib.load(selected_map_path).get_fdata()
        sum_of_maps += map_data
    
    averaged_map_data = sum_of_maps / len(split_maps_paths)
    averaged_map = nib.Nifti1Image(averaged_map_data, affine)
    averaged_map_filename = f"permuted_mean_group_accuracy_map_{iteration+1:03d}.nii"
    averaged_map_path = os.path.join(output_dir, averaged_map_filename)
    averaged_map.to_filename(averaged_map_path)
    
    averaged_map_mean_value = np.mean(averaged_map_data)
    
    # Finish the iteration
    end_time = time.time()
    duration = end_time - start_time
    
    return {"Iteration": iteration + 1, 
            "Duration (s)": duration,
            "Selected Maps": selected_maps,
            "mean_group_acc": averaged_map_mean_value}
    
    #print(f"Saved permuted group accuracy map for iteration {iteration+1}")
    
# Measure the total processing time
total_start_time = time.time()

# Use ProcessPoolExecutor to run iterations in parallel
iteration_timings = []

with ProcessPoolExecutor() as executor:
    # Start all iterations and return futures
    futures = [executor.submit(perform_iteration, iteration, output_dir) for iteration in range(n_iterations)]
    
    # As each future completes, record its duration
    for future in as_completed(futures):
        iteration_timings.append(future.result())

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total processing time: {total_duration:.2f} seconds.")

# Add total processing time to the timings list
iteration_timings.append({"Iteration": "Total", "Duration (s)": total_duration})

# Save timings to a CSV file
timings_file = os.path.join(output_dir, "timings.csv")
with open(timings_file, 'w', newline='') as file:
    # Define fieldnames that match the timing information
    fieldnames = ["Iteration", "Duration (s)"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    # Exclude 'mean_group_acc' and 'Selected Maps' from the data to be written
    for result in iteration_timings:
        if "Iteration" in result and "Duration (s)" in result:
            writer.writerow({key: result[key] for key in fieldnames})

# Save the collected data to a CSV file, corrected for mean selected maps
maps_csv_file_path = os.path.join(output_dir, "null_distribution_group_mean.csv")
with open(maps_csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Iteration', 'Selected Maps', 'Mean Group Acc']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in iteration_timings:
        if "Selected Maps" in result:
            # Process each path in selected_maps to extract only the last two parts
            processed_selected_maps = [
                '/'.join(path.split(os.sep)[-2:])  # Adjusted to include only the last two parts of path for context
                for path in result['Selected Maps']
            ]
            # Convert the list of processed paths to a string
            selected_maps_str = '|'.join(processed_selected_maps)
            prepared_data = {
                'Iteration': result['Iteration'],
                'Selected Maps': selected_maps_str,
                'Mean Group Acc': result.get('mean_group_acc', '')
            }
            writer.writerow(prepared_data)

print(f"Timings saved to {timings_file}")


In [2]:
null_distribution = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut/permu_mean_group_acc_maps/null_distribution_group_mean.csv"

null_distribution_df = pd.read_csv(null_distribution)

# Access the "Mean Group Acc" column
null_distribution_df = null_distribution_df['Mean Group Acc']

# Count the number of rows
num_rows = null_distribution_df.count()
print(f"Number of rows: {num_rows}")

# check mean values of null distributions (should be close to 0.5)


# Compute the mean of the "Mean Group Acc" column
column_mean = null_distribution_df.mean()
print(f"Mean of 'Mean Group Acc': {column_mean}")


Number of rows: 100000
Mean of 'Mean Group Acc': -0.4120553650748447


# Generating figures of sinificant brain regions from SnPM


In [ ]:
result_dir = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level'
fname = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/lP_FDR+.img'

#brain_nii = nb.load(result_dir + '/lP_FWE+.img')
img = nb.load(fname)
nb.save(img, fname.replace('.img', '.nii'))
display = plotting.plot_glass_brain(None, display_mode='lyrz')
color = 'r'
display.add_contours(img, filled=True, levels=[1], colors=color)
display.title('regions uncovered by ISPA, cluster-level FWE')
#display.savefig(result_dir + '/{}_{}.png'.format(dataset_name, decoding_type))

In [ ]:

cluster_snpm = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/SnPM_filtered.nii'
cluster_snpm = nb.load(cluster_snpm)

vmax = -np.log10(1 / 10000)  # ~= -np.log10(1 / n_perm)


plotting.plot_stat_map(cluster_snpm,
                       cut_coords=(-9, 0, 5, 10),
                      title='SnPM FWE cluster mass', vmax=vmax, display_mode='x', cmap = "Oranges",
                      black_bg=False, draw_cross = False, dim = 1, alpha=0.85)

In [ ]:

cluster_snpm = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/SnPM_filtered.nii'
cluster_snpm = nb.load(cluster_snpm)

vmax = -np.log10(1 / 10000)  # ~= -np.log10(1 / n_perm)


plotting.plot_stat_map(cluster_snpm,
                       cut_coords=(-20,-15,-9,5),
                     vmax=vmax, display_mode='z', cmap = "Oranges",
                      black_bg=False, draw_cross = False, dim = 1, alpha=0.85)